<a href="https://colab.research.google.com/github/dotsnangles/Retrieval-Based-Chatbot/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !rm -rf /content/Poly-Encoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Tue Jul  5 02:50:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q transformers datasets folium==0.2.1 apex

In [ ]:
!git clone https://github.com/dotsnangles/Poly-Encoder.git

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 102 (delta 24), reused 15 (delta 6), pack-reused 57
Receiving objects: 100% (102/102), 34.60 KiB | 164.00 KiB/s, done.
Resolving deltas: 100% (49/49), done.


In [ ]:
%cd Poly-Encoder

/content/Poly-Encoder


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoModel, AutoTokenizer, AutoConfig, BertPreTrainedModel, BertModel, BertConfig, BertTokenizer

import os
import time
import json
import shutil
import argparse
import numpy as np
from tqdm import tqdm
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

from transformers import AutoModel, AutoTokenizer, AutoConfig, BertPreTrainedModel, BertModel, BertConfig, BertTokenizer, BertTokenizerFast
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

from dataset import SelectionDataset
from transform import SelectionSequentialTransform, SelectionJoinTransform, SelectionConcatTransform
from encoder import PolyEncoder, BiEncoder, CrossEncoder

from sklearn.metrics import label_ranking_average_precision_score

import logging

In [ ]:
%cd /content/Poly-Encoder

/content/Poly-Encoder


In [ ]:
logging.basicConfig(level=logging.ERROR)

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

def eval_running_model(dataloader, test=False):
    model.eval()
    eval_loss, eval_hit_times = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    r10 = r2 = r1 = r5 = 0
    mrr = []
    for step, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        if args.architecture == 'cross':
            text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch, labels_batch = batch
            with torch.no_grad():
                logits = model(text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch)
                loss = F.cross_entropy(logits, torch.argmax(labels_batch, 1))
        else:
            context_token_ids_list_batch, context_input_masks_list_batch, \
            response_token_ids_list_batch, response_input_masks_list_batch, labels_batch = batch
            with torch.no_grad():
                logits = model(context_token_ids_list_batch, context_input_masks_list_batch,
                                              response_token_ids_list_batch, response_input_masks_list_batch)
                loss = F.cross_entropy(logits, torch.argmax(labels_batch, 1))
        r2_indices = torch.topk(logits, 2)[1] # R 2 @ 100
        r5_indices = torch.topk(logits, 5)[1] # R 5 @ 100
        r10_indices = torch.topk(logits, 10)[1] # R 10 @ 100
        r1 += (logits.argmax(-1) == 0).sum().item()
        r2 += ((r2_indices==0).sum(-1)).sum().item()
        r5 += ((r5_indices==0).sum(-1)).sum().item()
        r10 += ((r10_indices==0).sum(-1)).sum().item()
        # mrr
        logits = logits.data.cpu().numpy()
        for logit in logits:
            y_true = np.zeros(len(logit))
            y_true[0] = 1
            mrr.append(label_ranking_average_precision_score([y_true], [logit]))
        eval_loss += loss.item()
        nb_eval_examples += labels_batch.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = r1 / nb_eval_examples
    if not test:
        result = {
            'train_loss': tr_loss / nb_tr_steps,
            'eval_loss': eval_loss,
            'R1': r1 / nb_eval_examples,
            'R2': r2 / nb_eval_examples,
            'R5': r5 / nb_eval_examples,
            'R10': r10 / nb_eval_examples,
            'MRR': np.mean(mrr),
            'epoch': epoch,
            'global_step': global_step,
        }
    else:
        result = {
            'eval_loss': eval_loss,
            'R1': r1 / nb_eval_examples,
            'R2': r2 / nb_eval_examples,
            'R5': r5 / nb_eval_examples,
            'R10': r10 / nb_eval_examples,
            'MRR': np.mean(mrr),
        }

    return result

In [ ]:
!gdown -q --folder 1Ipr-aNF5ELMY0HTXAmeV26LlgktKUfmG

In [ ]:
args = {
        "bert_model": 'klue/bert-base',
        "eval": False,
        "model_type": 'bert',
        "output_dir": 'chatbot_output/',
        "train_dir": './',

        "use_pretrain": True,
        "architecture": 'poly',

        "max_contexts_length": 128,
        "max_response_length": 32,
        "train_batch_size": 32,
        "eval_batch_size": 32,
        "print_freq": 100,

        "poly_m": 16,

        "learning_rate": 5e-5,
        "weight_decay": 0.01,
        "warmup_steps": 100,
        "adam_epsilon": 1e-8,
        "max_grad_norm": 1.0,

        "num_train_epochs": 10.0,
        'seed': 12345,
        'gradient_accumulation_steps': 1,
        "fp16": False,
        "fp16_opt_level": "O1",
        'gpu': 0
        }

In [ ]:
from easydict import EasyDict as edict
args = edict(args)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "%d" % args.gpu
set_seed(args)

## init dataset and bert model
model_name = args.bert_model
tokenizer = BertTokenizer.from_pretrained(model_name)

context_transform = SelectionJoinTransform(tokenizer=tokenizer, max_len=args.max_contexts_length)
response_transform = SelectionSequentialTransform(tokenizer=tokenizer, max_len=args.max_response_length)
concat_transform = SelectionConcatTransform(tokenizer=tokenizer, max_len=args.max_response_length+args.max_contexts_length)

print('=' * 80)
print('Train dir:', args.train_dir)
print('Output dir:', args.output_dir)
print('=' * 80)

if not args.eval:
    train_dataset = SelectionDataset(os.path.join(args.train_dir, 'train_data_source.pickle'),
                                                                    context_transform, response_transform, concat_transform, sample_cnt=None, mode=args.architecture)
    val_dataset = SelectionDataset(os.path.join(args.train_dir, 'val_data_source.pickle'),
                                                                context_transform, response_transform, concat_transform, sample_cnt=None, mode=args.architecture)
    train_dataloader = DataLoader(train_dataset, batch_size=args.train_batch_size, collate_fn=train_dataset.batchify_join_str, shuffle=True, num_workers=0)
    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
else: # test
    val_dataset = SelectionDataset(os.path.join(args.train_dir, 'test_data_source.pickle'),
                                                                context_transform, response_transform, concat_transform, sample_cnt=None, mode=args.architecture)

val_dataloader = DataLoader(val_dataset, batch_size=args.eval_batch_size, collate_fn=val_dataset.batchify_join_str, shuffle=False, num_workers=0)


epoch_start = 1
global_step = 0
best_eval_loss = float('inf')
best_test_loss = float('inf')

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

log_wf = open(os.path.join(args.output_dir, 'log.txt'), 'a', encoding='utf-8')
print(args, file=log_wf)

state_save_path = os.path.join(args.output_dir, '{}_{}_pytorch_model.bin'.format(args.architecture, args.poly_m))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

########################################
## build BERT encoder
########################################

bert_config = BertConfig.from_pretrained(model_name)

bert = BertModel.from_pretrained(model_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=args.poly_m)

model.resize_token_embeddings(len(tokenizer))
model.to(device)

if args.eval:
    print('Loading parameters from', state_save_path)
    model.load_state_dict(torch.load(state_save_path))
    test_result = eval_running_model(val_dataloader, test=True)
    print (test_result)
    exit()

no_decay = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
)
if args.fp16:
    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

print_freq = args.print_freq//args.gradient_accumulation_steps
eval_freq = min(len(train_dataloader) // 2, 1000)
eval_freq = eval_freq//args.gradient_accumulation_steps
print('Print freq:', print_freq, "Eval freq:", eval_freq)

for epoch in range(epoch_start, int(args.num_train_epochs) + 1):
    tr_loss = 0
    nb_tr_steps = 0
    with tqdm(total=len(train_dataloader)//args.gradient_accumulation_steps) as bar:
        for step, batch in enumerate(train_dataloader):
            model.train()
            optimizer.zero_grad()
            batch = tuple(t.to(device) for t in batch)
            if args.architecture == 'cross':
                text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch, labels_batch = batch
                loss = model(text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch, labels_batch)
            else:
                context_token_ids_list_batch, context_input_masks_list_batch, \
                response_token_ids_list_batch, response_input_masks_list_batch, labels_batch = batch
                loss = model(context_token_ids_list_batch, context_input_masks_list_batch,
                                        response_token_ids_list_batch, response_input_masks_list_batch,
                                        labels_batch)

            loss = loss / args.gradient_accumulation_steps
            
            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            
            tr_loss += loss.item()

            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                nb_tr_steps += 1
                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step += 1

                if nb_tr_steps and nb_tr_steps % print_freq == 0:
                    bar.update(min(print_freq, nb_tr_steps))
                    time.sleep(0.02)
                    print(global_step, tr_loss / nb_tr_steps)
                    log_wf.write('%d\t%f\n' % (global_step, tr_loss / nb_tr_steps))

                if global_step and global_step % eval_freq == 0:
                    val_result = eval_running_model(val_dataloader)
                    print('Global Step %d VAL res:\n' % global_step, val_result)
                    log_wf.write('Global Step %d VAL res:\n' % global_step)
                    log_wf.write(str(val_result) + '\n')

                    if val_result['eval_loss'] < best_eval_loss:
                        best_eval_loss = val_result['eval_loss']
                        val_result['best_eval_loss'] = best_eval_loss
                        # save model
                        print('[Saving at]', state_save_path)
                        log_wf.write('[Saving at] %s\n' % state_save_path)
                        torch.save(model.state_dict(), state_save_path)

            log_wf.flush()

    # add a eval step after each epoch
    val_result = eval_running_model(val_dataloader)
    print('Epoch %d, Global Step %d VAL res:\n' % (epoch, global_step), val_result)
    log_wf.write('Global Step %d VAL res:\n' % global_step)
    log_wf.write(str(val_result) + '\n')

    if val_result['eval_loss'] < best_eval_loss:
        best_eval_loss = val_result['eval_loss']
        val_result['best_eval_loss'] = best_eval_loss
        # save model
        print('[Saving at]', state_save_path)
        log_wf.write('[Saving at] %s\n' % state_save_path)
        torch.save(model.state_dict(), state_save_path)
    print(global_step, tr_loss / nb_tr_steps)
    log_wf.write('%d\t%f\n' % (global_step, tr_loss / nb_tr_steps))

Train dir: ./
Output dir: chatbot_output/


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprec

Print freq: 100 Eval freq: 1000


  4%|▍         | 100/2555 [00:51<21:13,  1.93it/s]

100 16.05421730518341


...

Epoch 3, Global Step 7665 VAL res:
 {'train_loss': 2.7753557121217134, 'eval_loss': 2.708037655672923, 'R1': 0.06978557504873294, 'R2': 0.13157894736842105, 'R5': 0.33235867446393763, 'R10': 0.6697855750487329, 'MRR': 0.22163258281679338, 'epoch': 3, 'global_step': 7665}
7665 2.7753557121217134


  4%|▍         | 100/2555 [00:51<20:53,  1.96it/s]

7765 2.7743221974372863


  4%|▍         | 100/2555 [00:58<24:00,  1.70it/s]


KeyboardInterrupt: ignored

In [ ]:
class PolyEncoder(BertPreTrainedModel):
    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = kwargs['bert']
        self.poly_m = kwargs['poly_m']
        self.poly_code_embeddings = nn.Embedding(self.poly_m, config.hidden_size)
        # https://github.com/facebookresearch/ParlAI/blob/master/parlai/agents/transformer/polyencoder.py#L355
        torch.nn.init.normal_(self.poly_code_embeddings.weight, config.hidden_size ** -0.5)

    def dot_attention(self, q, k, v):
        # q: [bs, poly_m, dim] or [bs, res_cnt, dim]
        # k=v: [bs, length, dim] or [bs, poly_m, dim]
        attn_weights = torch.matmul(q, k.transpose(2, 1)) # [bs, poly_m, length]
        attn_weights = F.softmax(attn_weights, -1)
        output = torch.matmul(attn_weights, v) # [bs, poly_m, dim]
        return output

    def forward(self, context_input_ids, context_input_masks,
                            responses_input_ids, responses_input_masks, labels=None, emb_sample=None):
        # during training, only select the first response
        # we are using other instances in a batch as negative examples
        if labels is not None:
            responses_input_ids = responses_input_ids[:, 0, :].unsqueeze(1)
            responses_input_masks = responses_input_masks[:, 0, :].unsqueeze(1)
        batch_size, res_cnt, seq_length = responses_input_ids.shape # res_cnt is 1 during training

        # context encoder
        ctx_out = self.bert(context_input_ids, context_input_masks)[0]  # [bs, length, dim]
        poly_code_ids = torch.arange(self.poly_m, dtype=torch.long).to(context_input_ids.device)
        poly_code_ids = poly_code_ids.unsqueeze(0).expand(batch_size, self.poly_m)
        poly_codes = self.poly_code_embeddings(poly_code_ids) # [bs, poly_m, dim]
        embs = self.dot_attention(poly_codes, ctx_out, ctx_out) # [bs, poly_m, dim]

        # response encoder
        responses_input_ids = responses_input_ids.view(-1, seq_length)
        responses_input_masks = responses_input_masks.view(-1, seq_length)
        cand_emb = self.bert(responses_input_ids, responses_input_masks)[0][:,0,:] # [bs, dim]
        cand_emb = cand_emb.view(batch_size, res_cnt, -1) # [bs, res_cnt, dim]

        # merge
        if labels is not None:
            # we are recycling responses for faster training
            # we repeat responses for batch_size times to simulate test phase
            # so that every context is paired with batch_size responses
            cand_emb = cand_emb.permute(1, 0, 2) # [1, bs, dim]
            cand_emb = cand_emb.expand(batch_size, batch_size, cand_emb.shape[2]) # [bs, bs, dim]
            ctx_emb = self.dot_attention(cand_emb, embs, embs).squeeze() # [bs, bs, dim]
            dot_product = (ctx_emb*cand_emb).sum(-1) # [bs, bs]
            mask = torch.eye(batch_size).to(context_input_ids.device) # [bs, bs]
            loss = F.log_softmax(dot_product, dim=-1) * mask
            loss = (-loss.sum(dim=1)).mean()
            return loss

        # from DB
        elif emb_sample is not None:

            cand_emb = emb_sample[0][:,0,:] # [bs, dim]
            cand_emb = cand_emb.view(batch_size, res_cnt, -1) # [bs, res_cnt, dim]
            ctx_emb = self.dot_attention(cand_emb, embs, embs) # [bs, res_cnt, dim]
            dot_product = (ctx_emb*cand_emb).sum(-1)
            return dot_product

        else:
            ctx_emb = self.dot_attention(cand_emb, embs, embs) # [bs, res_cnt, dim]
            dot_product = (ctx_emb*cand_emb).sum(-1)
            return dot_product

    def creat_emb(self, input_id, input_mask):
        

In [ ]:
!gdown --folder -q 1Ipr-aNF5ELMY0HTXAmeV26LlgktKUfmG

In [ ]:
import pickle

with open('/content/감성대화챗봇데이터/train_data_source.pickle', 'rb') as f:
    train = pickle.load(f)

In [ ]:
train[0]

{'context': ['participant 1: 아내가 드디어 출산하게 되어서 정말 신이 나.',
  'participant 2: 아내분이 출산을 하시는군요. 정말 축하드려요.',
  'participant 1: 아 지금 정말 신이 나.',
  'participant 2: 잘 된 일이네요.',
  'participant 1: 아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지. '],
 'labels': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'responses': ['좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.',
  '고마운 직원들에게 선물을 돌리려고 하시는군요.',
  '그거 좋은 생각이네요. 무슨 이야기를 하실 건가요?',
  '열심히 살았는데 건강검진 결과가 좋지 않을까봐 허무하시군요.',
  '은퇴를 생각하니 흥분되시는군요.',
  '부장님과 잘 의논해서 잘 해결되시기를 바랄게요.',
  '혹시 다른 사람들도 그렇게 생각하고 있을까 봐 고민이 된 거군요.',
  '세탁비를 받으셔서 깨끗하게 입으시면 좋겠네요.',
  '요일을 정해서 육아를 하실 생각이시군요.',
  '친구에게 전화를 해보려는군요.',
  '있는 그대로의 모습을 보여주어 지금의 상황을 잘 극복하길 바라요.',
  '아빠와의 고민상담을 통해 원하는 결과가 있었으면 좋겠어요.',
  '가족과 함께 해결책을 찾아 보려 하시는군요.',
  '맛있는 음식 드시고 기분 전환을 잘했으면 좋겠어요.',
  '친구에게 사정이 있을 거로 생각하고 기다리고 계시지만 한편으로는 불안하기도 하신 거군요.',
  '혼자서도 여가 활동을 즐기실 수 있을 만큼 건강이 회복되면 좋겠어요.']}

In [ ]:
import pickle

with open('/content/감성대화챗봇데이터/val_data_source.pickle', 'rb') as f:
    dev = pickle.load(f)

In [ ]:
dev

[{'context': ['participant 1: 요즘 부모님과 많이 부딪혀.',
   'participant 2: 어떤 일로 부딪히시나요?',
   'participant 1: 난 하고 싶은 일이 있는데 부모님은 안정적인 일만 하라고 하셔. 내 마음대로 하지도 못해.',
   'participant 2: 부모님과의 충돌로 힘드시겠군요.',
   'participant 1: 응. 그래도 난 내가 하고 싶은 일을 위해 노력할 거야.'],
  'labels': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'responses': ['자신이 하고 싶은 일을 목표로 하기로 하셨군요.',
   '골프를 배워 보려고 하시는군요.',
   '경제 활동을 늘리려고 하시는군요.',
   '부디 좋은 결과가 이어질 수 있도록 이성적인 대화를 하셨으면 좋겠어요.',
   '힘드시겠어요. 모욕적인 행동을 했다니 정말 자존심이 많이 상했겠네요.',
   '아내분과 잘 대화해서 긍정적인 결과를 얻길 바라요.',
   '마음이 따뜻하시네요. 가족들이 모두 강아지를 잘 돌보게 되면 좋겠어요.',
   '마음대로 음식물을 드실수 있어서 기쁘시군요.',
   '맞아요. 떳떳하게 다니시고 그러면 따돌림이라는 말이 언제 그랬냐 싶을 거예요.',
   '험담을 한 적이 없으신데 억울하시겠어요.',
   '혼자 밥 먹는 시간이 너무 싫어서 밥 먹는 시간이 오지 않았으면 좋겠다고 생각하시는군요.',
   '감당할 수 없을 정도의 업무량으로 힘이 많이 드시나 보군요.',
   '건강이 안 좋아 고민이 많으시군요.',
   '일에 집중하느라 건강 관리에 소홀하셨군요.',
   '남자친구분의 좋은 점을 찾고 긍정적으로 생각하시는 점 너무 멋지세요.',
   '상대방을 이해하는 데 어려움을 겪고 계시군요.']},
 {'context': ['participant 1: 엄마가 결국 집을 나갔어. 너무 너무 슬퍼.',
   'participant 2: 어머니께서 